In [4]:
pip install dash-bootstrap-components

In [ ]:
# définir le répertoire du projet contenant le dossier data/ et notebooks/
HOME = Path.cwd().parent
print(f"Home directory: {HOME}")

# définir le répertoire des données
DATA = Path(HOME, "data")
print(f"Data directory: {DATA}")

SyntaxError: invalid syntax (895532910.py, line 2)

In [9]:
# Importation des bibliothèques
from pathlib import Path
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc

# Charger les données
data = pd.read_csv(Path(DATA, "2016_Building_Energy_2.csv"), sep=",") # Remplacer par ton fichier de données

# Initialiser l'application Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Layout du Dashboard
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col(html.H1("Dashboard - Consommation d'Énergie des Bâtiments"), width=12)
    ], className="mb-4"),

    # Filtre interactif
    dbc.Row([
        dbc.Col([
            html.Label("Filtrer par année de construction :"),
            dcc.RangeSlider(
                id='year-slider',
                min=data['YearBuilt'].min(),
                max=data['YearBuilt'].max(),
                step=1,
                marks={year: str(year) for year in range(data['YearBuilt'].min(), data['YearBuilt'].max(), 10)},
                value=[data['YearBuilt'].min(), data['YearBuilt'].max()]
            )
        ], width=6),

        dbc.Col([
            html.Label("Sélectionner le type de bâtiment :"),
            dcc.Dropdown(
                id='building-type-dropdown',
                options=[{'label': bld, 'value': bld} for bld in data['LargestPropertyUseTypeGFA'].unique()],
                multi=True,
                placeholder="Sélectionner un ou plusieurs types"
            )
        ], width=6)
    ], className="mb-4"),

    # Graphiques
    dbc.Row([
        dbc.Col(dcc.Graph(id='energy-usage-graph'), width=6),
        dbc.Col(dcc.Graph(id='ghg-emissions-graph'), width=6)
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id='scatter-plot-energy'), width=12)
    ])
], fluid=True)

# Callback pour mettre à jour les graphiques
@app.callback(
    [Output('energy-usage-graph', 'figure'),
     Output('ghg-emissions-graph', 'figure'),
     Output('scatter-plot-energy', 'figure')],
    [Input('year-slider', 'value'),
     Input('building-type-dropdown', 'value')]
)
def update_graphs(year_range, building_types):
    # Filtrer les données
    filtered_data = data[(data['YearBuilt'] >= year_range[0]) & (data['YearBuilt'] <= year_range[1])]
    if building_types:
        filtered_data = filtered_data[filtered_data['LargestPropertyUseTypeGFA'].isin(building_types)]

    # Graphique 1 : Consommation d'énergie totale par type de bâtiment
    fig_energy = px.bar(
        filtered_data.groupby('LargestPropertyUseTypeGFA')['SiteEnergyUse(kBtu)'].sum().reset_index(),
        x='LargestPropertyUseTypeGFA',
        y='SiteEnergyUse(kBtu)',
        title="Consommation Totale d'Énergie par Type de Bâtiment",
        labels={'SiteEnergyUse(kBtu)': 'Consommation (kBtu)'},
        color='LargestPropertyUseTypeGFA'
    )

    # Graphique 2 : Émissions de gaz à effet de serre
    fig_ghg = px.bar(
        filtered_data.groupby('LargestPropertyUseTypeGFA')['TotalGHGEmissions'].sum().reset_index(),
        x='LargestPropertyUseTypeGFA',
        y='TotalGHGEmissions',
        title="Émissions Totales de GES par Type de Bâtiment",
        labels={'TotalGHGEmissions': 'GES (tonnes)'},
        color='LargestPropertyUseTypeGFA'
    )

    # Graphique 3 : Scatter Plot entre consommation et émissions
    fig_scatter = px.scatter(
        filtered_data,
        x='SiteEnergyUse(kBtu)',
        y='TotalGHGEmissions',
        size='PropertyGFATotal',
        color='LargestPropertyUseTypeGFA',
        hover_name='LargestPropertyUseTypeGFA',
        title="Relation entre Consommation d'Énergie et Émissions de GES",
        labels={'SiteEnergyUse(kBtu)': 'Consommation (kBtu)', 'TotalGHGEmissions': 'GES (tonnes)'}
    )

    return fig_energy, fig_ghg, fig_scatter

# Lancer le serveur
if __name__ == '__main__':
    app.run_server(debug=True)


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\HP\\Desktop\\ISE2_2024-2025\\Environnement python\\data\\2016_Building_Energy_2.csv'